# C3.2. Тонкости обработки исключений. Собственные классы исключений.

Классы +-- SystemExit +-- KeyboardInterrupt +-- GeneratorExit — являются исключениями, которые нельзя поймать, т.к. их возникновение не зависит от выполнения программы. А все, что наследуются от Exception, можно отловить и обработать

In [1]:
try:
    raise ZeroDivisionError  # возбуждаем исключение ZeroDivisionError
except ArithmeticError:  # ловим его родителя
    print("Hello from arithmetic error")

Hello from arithmetic error


 если вы хотите ловить сначала сами классы, а потом любой другой из наследуемых, то убедитесь, что вы идёте от конкретного к абстрактному.

Вот правильный пример, для наглядности:

In [2]:
try:
    raise ZeroDivisionError
except ZeroDivisionError:  # сначала пытаемся поймать наследника
    print("Zero division error")
except ArithmeticError:  # потом ловим потомка
    print("Arithmetic error")

Zero division error


In [3]:
# Принцип написания и отлова собственного исключения следующий:

class MyException(Exception):  # создаём пустой класс – исключения
    pass
try:
    raise MyException("message")  # поднимаем наше исключение
except MyException as e:  # ловим его за хвост как шкодливого котёнка
    print(e)  # выводим информацию об исключении


message


In [ ]:
# собственные исключения с наследованием:

# создаём пустой класс – исключения потомка, наследуемся от exception
class ParentException(Exception):
    pass

# создаём пустой класс – исключение наследника, наследуемся от ParentException
class ChildException(ParentException):
    pass

try:
    raise ChildException("message")  # поднимаем исключение-наследник
except ParentException as e:  # ловим его родителя
    print(e)  # выводим информацию об исключении


класс с самописным исключением не обязательно должен быть пустым. Если вы хотите добавить собственные аргументы в конструктор, дополнительно произвести какие-либо операции, то можете спокойно это делать, главное — не забыть о нескольких нюансах:

In [ ]:
class ParentException(Exception):
    # допишем к нашему пустому классу конструктор, который будет печатать дополнительно в консоль информацию об ошибке.
    def __init__(self, message, error):
        super().__init__(message)  # помним про вызов конструктора родительского класса
        print(f"Errors: {error}")  # печатаем ошибку


# создаём пустой класс – исключение наследника, наследуемся от ParentException
class ChildException(ParentException):
    def __init__(self, message, error):
        super().__init__(message, error)


try:
    # поднимаем исключение-наследник, передаём дополнительный аргумент
    raise ChildException("message", "error")
except ParentException as e:
    print(e)  # выводим информацию об исключении
# Сначала мы увидим то, что напишет нам конструктор родительского класса, а потом уже увидим сообщение об ошибке.


### ***Давайте подведём итоги:***

- Исключения — это такие особенные классы, 
    которые как и любые классы можно наследовать. 
    Если вы хотите ловить несколько исключений, 
    то сначала ловите потомков, а потом родителей, 
    чтобы ничего не упустить.
- Чтобы создать собственный класс, нужно просто написать пустой класс 
    и наследовать его от класса Exception, этого будет достаточно.
- Не обязательно отлавливать сам класс, при необходимости можно 
    отлавливать его родителя, это тоже будет работать, 
    но вы можете упустить важную информацию.


Задание 3.2.5

Задание на самопроверку.

Создать класс Square. Добавить в конструктор класса Square собственное исключение NonPositiveDigitException, унаследованное от ValueError, которое будет срабатывать каждый раз, когда сторона квадрата меньше или равна 0.


In [6]:
class NonPositiveDigitException(ValueError):
    def __str__(self):
        return "сторона квадрата меньше или равна 0"

class Square:
    def __init__(self, side):
        self.side = side
    def get_area(self):
        try:
            if self.side <= 0:
                raise NonPositiveDigitException
            else: 
                return self.side**2
        except ValueError as e:
            print(e)

s = Square(0)
print("s area = " + str(s.get_area()))

сторона квадрата меньше или равна 0
s area = None


In [10]:
# Эталон решения

class NonPositiveDigitException(ValueError):
    pass

class Square:
    def __init__(self, a):
        if a <= 0:
            raise NonPositiveDigitException(
                'Неправильно указанна сторона квадрата')

s = Square(1)

# C3.3. Работа с импортом

**Как импортировать модуль**

    import sys
    import os

Импорт происходит с помощью зарезервированного слова **import** название **модуля**.

Для того, чтобы обратиться к содержимому модуля, нужно после его имени поставить точку и указать нужный вам элемент.

    import os
    
    print(os.getcwd())  # получить текущую директорию
    print(os.listdir())  # получить список файлов текущей директории

*Не следует импортировать модули в одну строку, каждый отдельный модуль должен импортировать на отдельной строке:*

    # правильно
    import os
    import sys
    
    # неправильно
    import os, sys

Но если вы хотите импортировать, например, несколько функций из одного модуля, можно перечислить их через запятую:

    from subprocess import Popen, PIPE

Также для импорта существует несколько различных способов записи, но делают они тоже самое: импорт всего модуля и обращение к функции через имя модуля:

    import math
    print(math.pi)  # 3.141592653589793

Импортируем всё из модуля, это позволяет сократить запись, и обращаться к функциям, не указывая имя модуля:

    from math import *  # импортируем всё из модуля math
    print(pi)  # 3.141592653589793

Даем новое имя модулю, это позволяет избежать ошибок, если у вас есть функции, которая называется так же как модуль, то вы можете дать новое имя модулю для данного скрипта:

    import math as m  # использование нового имени для обращения к импортированному модулю
    print(m.pi)  # 3.141592653589793

Даем новое имя функции или переменной, это позволяет избежать ошибок, если у вас есть одноименная функция в вашей программе, и вы не хотите её переименовать:

    from math import pi as PI
    print(PI)  # 3.141592653589793





## Как правильно составить модуль?

1. Вся основная логика модуля заключена в отдельные функции или классы. На глобальном уровне могут быть объявлены только константы или необходимые для инициализации модуля операции.

2. Если вы планируете, что модуль могут запускать как самостоятельный скрипт – используйте следующую инструкцию: 

            if __name__ == '__main__':.

    *Как это работает?* Ваш скрипт может выполняться и самостоятельно, а может быть импортирован как модуль другим скриптом. Чтобы выделить код, который не должен выполняться при импорте его следует поместить в условный оператор с условием 
  
            if __name__ == '__main__':.
        
3. Хорошая структура модуля выглядит следующим образом:                                                                                                                         

- Docstring (описание) модуля.
- Область импорта:
  - импорты системных библиотек;
  - импорты стандартных пакетов (из PyPI);
  - импорты ваших модулей (локальных).
- Область объявление глобальных констант.
- Инициализация модуля.
- Область определения функций и классов.
- Функции.

                if __name__ == '__main__' 
                
  (метод main) по желанию (это одни из немногих нюансов при работе с собственными модулями).

***ВАЖНО***: *Чтобы модули заработали правильно, их нужно хранить в той же папке, в которой вы запускаете главный скрипт, иначе Python не найдёт ваш модуль!*


In [14]:
import math
import time
f = math.trunc(math.fmod(math.fabs(-10000000), 55)+0.3)
f
t = time.asctime()
t

'Sun Jul 25 02:17:50 2021'

In [18]:
import time

i = 10
while i != 0:
  print(i)
  i -= 1
  time.sleep(1)
print("Time's up!")



10
9
8
7
6
5
4
3
2
1
Time's up!


## **Задание 3.3.7**

*Задание на самопроверку.*

Вам нужно написать два модуля:

+ Первый должен содержать число Пи в виде константы 3.14 и две функции, которые будут считать площадь круга и прямоугольника.
+ Второй модуль должен импортировать первый, далее запрашивать у пользователя размеры круга и квадрата. В результате выводить, какая из фигур больше.


In [ ]:
PI = 3.14


def circle_area(r):
   return PI * (r ** 2)


def rect_area(a, b):
   return a * b


if __name__ == '__main__':
   # проверяем работоспособность функции, дальнейшая часть не будет импортирована
   # если ответы будут отличаться, то будет вызвана ошибка
   assert circle_area(5) == 78.5
   assert rect_area(5, 4) == 20

##############################



In [ ]:
from module_name import *


def main():
   r = input('Введите радиус круга:\n')

   a = input('Введите длину прямоугольника:\n')
   b = input('Введите ширину прямоугольника:\n')

   if circle_area(r) > rect_area(a, b):
       print('Площадь круга больше')
   else:
       print('Площадь прямоугольника больше')
